## Import

Load data.

In [1]:
import os
import pandas as pd
from datetime import date
from utils import *

save = True
delim = "|"
timestamp = date.today().strftime("%Y%m%d")
in_path = os.path.join("input", "manual_input.csv")
out_path = os.path.join("output", f"manual_input_{timestamp}.csv")
audio_dir = os.path.join("output", "audio")

cols = ["Simp", "Trad", "Meaning", "Simp (Color)", "Trad (Color)", "Pinyin (Color)", "Sound", "Silhouette"]
df = pd.read_csv(in_path, dtype = str)
df.columns = ["Simp", "Trad", "Pinyin (Color)", "Meaning"]
df = df[[col for col in cols if col in df.columns]]
df_edited = df.copy()

print("Shape:", df.shape)
df.head()

Shape: (5, 4)


,Simp,Trad,Meaning,Pinyin (Color)
0,向北延伸至加拿大,向北延伸至加拿大,to extend north toward Canada,xiàng běi yánshēn zhì jiānádà
1,延伸,延伸,to extend,yánshēn
2,平板,平板,tablet (computer),píngbǎn
3,强大,強大,powerful; formidable; strong,qiángdà
4,防守,防守,defense; to defend,fángshǒu


# Basic Formatting

In [2]:
df_edited["Meaning"] = df["Meaning"].apply(lambda x : x.replace("; ", "<br>"))
df_edited["Silhouette"] = df["Simp"].astype(str).apply(get_silhouette)
df_edited = df_edited[[col for col in cols if col in df_edited.columns]]

print("\nView table:")
df_edited


View table:


,Simp,Trad,Meaning,Pinyin (Color),Silhouette
0,向北延伸至加拿大,向北延伸至加拿大,to extend north toward Canada,xiàng běi yánshēn zhì jiānádà,_ _ _ _ _ _ _ _
1,延伸,延伸,to extend,yánshēn,_ _
2,平板,平板,tablet (computer),píngbǎn,_ _
3,强大,強大,powerful<br>formidable<br>strong,qiángdà,_ _
4,防守,防守,defense<br>to defend,fángshǒu,_ _


## Coloring

Convert characters to pinyin. Then add color.

In [3]:
pin_split = (
    df_edited["Pinyin (Color)"]
        .apply(PINYIN_RE.split)
        .apply(delim.join)
        .apply(full_split)
)
simp_split = df_edited["Simp"].apply(delim.join).apply(full_split, args = delim)
trad_split = df_edited["Trad"].apply(delim.join).apply(full_split, args = delim)

df_edited["Simp (Color)"] = pd.concat([pin_split, simp_split], axis = 1).apply(lambda row: add_color_hanzi(row[0], row[1]), axis = 1)
df_edited["Trad (Color)"] = pd.concat([pin_split, trad_split], axis = 1).apply(lambda row: add_color_hanzi(row[0], row[1]), axis = 1)
df_edited["Pinyin (Color)"] = pin_split.apply(simplify_split).apply(tag_split)

df_edited = df_edited[[col for col in cols if col in df_edited.columns]]
df_edited.head()

,Simp,Trad,Meaning,Simp (Color),Trad (Color),Pinyin (Color),Silhouette
0,向北延伸至加拿大,向北延伸至加拿大,to extend north toward Canada,<span class = 'tone4'>向</span><span class = 't...,<span class = 'tone4'>向</span><span class = 't...,<span class = 'tone4'>xiàng </span><span class...,_ _ _ _ _ _ _ _
1,延伸,延伸,to extend,<span class = 'tone2'>延</span><span class = 't...,<span class = 'tone2'>延</span><span class = 't...,<span class = 'tone2'>yán</span><span class = ...,_ _
2,平板,平板,tablet (computer),<span class = 'tone2'>平</span><span class = 't...,<span class = 'tone2'>平</span><span class = 't...,<span class = 'tone2'>píng</span><span class =...,_ _
3,强大,強大,powerful<br>formidable<br>strong,<span class = 'tone2'>强</span><span class = 't...,<span class = 'tone2'>強</span><span class = 't...,<span class = 'tone2'>qiáng</span><span class ...,_ _
4,防守,防守,defense<br>to defend,<span class = 'tone2'>防</span><span class = 't...,<span class = 'tone2'>防</span><span class = 't...,<span class = 'tone2'>fáng</span><span class =...,_ _


In [4]:
df_edited[df_edited["Simp (Color)"] == "(color error)"]

,Simp,Trad,Meaning,Simp (Color),Trad (Color),Pinyin (Color),Silhouette


# Sound

In [5]:
df_edited["Sound"] = df_edited["Simp"].apply(lambda s : save_audio(s, audio_dir, save))
df_edited = df_edited[[col for col in cols if col in df_edited.columns]]

errors = df_edited["Sound"][df_edited["Sound"].apply(lambda s : s.startswith("FAILED"))]
print("Errors:\n", errors)
df_edited.head()

Errors:
 Series([], Name: Sound, dtype: object)


C:\Users\micha\AppData\Local\Temp\ipykernel_11428\845422964.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_edited["Sound"] = df_edited["Simp"].apply(lambda s : save_audio(s, audio_dir, save))


,Simp,Trad,Meaning,Simp (Color),Trad (Color),Pinyin (Color),Sound,Silhouette
0,向北延伸至加拿大,向北延伸至加拿大,to extend north toward Canada,<span class = 'tone4'>向</span><span class = 't...,<span class = 'tone4'>向</span><span class = 't...,<span class = 'tone4'>xiàng </span><span class...,[sound:向北延伸至加拿大.mp3],_ _ _ _ _ _ _ _
1,延伸,延伸,to extend,<span class = 'tone2'>延</span><span class = 't...,<span class = 'tone2'>延</span><span class = 't...,<span class = 'tone2'>yán</span><span class = ...,[sound:延伸.mp3],_ _
2,平板,平板,tablet (computer),<span class = 'tone2'>平</span><span class = 't...,<span class = 'tone2'>平</span><span class = 't...,<span class = 'tone2'>píng</span><span class =...,[sound:平板.mp3],_ _
3,强大,強大,powerful<br>formidable<br>strong,<span class = 'tone2'>强</span><span class = 't...,<span class = 'tone2'>強</span><span class = 't...,<span class = 'tone2'>qiáng</span><span class ...,[sound:强大.mp3],_ _
4,防守,防守,defense<br>to defend,<span class = 'tone2'>防</span><span class = 't...,<span class = 'tone2'>防</span><span class = 't...,<span class = 'tone2'>fáng</span><span class =...,[sound:防守.mp3],_ _


# Save

In [6]:
if save:
    df_edited.to_csv(out_path, index = False, header = False)